In [ ]:
import csv
import json

In [ ]:
pmc_id_list = []

with open('./pubmed_pmc.csv') as pmc:
    reader = csv.reader(pmc)
    for row in reader:
        # check pmc_row data
        if row[1] != 'False':
            pmc_id_list.append(row[1])


In [ ]:
pmc_id_list

In [ ]:
def get_valid_articles(lst):
    rst = []
    no_file =[]
    for i in lst:
        file_pmc_id = i
        file_name = 'article_' + file_pmc_id[3:] + '.json'
        file_path = '/home/luhan/articles_json/' + file_name

        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
        except:
            no_file.append(i)
            continue
        
        raw = data.get('full_text')
        
        if raw is None or len(raw) == 0:
            continue
        rst.append(i)
    return rst, no_file

In [ ]:
valid_pmc_id_list, notfound = get_valid_articles(pmc_id_list)

In [ ]:
valid_pmc_id_list

In [ ]:
len(notfound)

In [ ]:
len(valid_pmc_id_list)

In [ ]:
pairs_path = '/home/luhan/pairs/QnA_generation_result'

In [ ]:
from json import JSONDecodeError
import re

def build_pairs():
    pairs = []
    for i in valid_pmc_id_list:
        file_name = 'QnA_generation_' + i + '.json'
        file_path = pairs_path + '/' + file_name
    
        with open(file_path, 'r') as f:
            s = f.read()
            data = re.sub(r'\,(?=\s*?[\}\]])', '', s)
            data = json.loads(data)
            
        for indx, d in enumerate(data):
            if 'pmc_id' in d and d['pmc_id'] != i[3:]:
                print(f'PMC Error: {i}')
                print(data)
                raise ValueError
            else:
                d['pmc_id'] = i[3:]
                
            if 'chunk_id' in d and d['chunk_id'] != indx:
                break
            else:
                d['chunk_id'] = indx
        pairs.extend(data)
    return pairs

In [ ]:
pairs = build_pairs()

In [ ]:
with open('valid_pairs.json', 'w') as f:
    json.dump(pairs, f)

In [ ]:
len(pairs)

In [ ]:
import os

# Replace 'your_directory_path' with the path to your target directory
directory_path = '/home/luhan/pairs/QnA_generation_result'

# List all entries in the given directory
entries = os.listdir(directory_path)

# Count how many of those entries are files
file_count = sum(os.path.isfile(os.path.join(directory_path, entry)) for entry in entries)

print(f'There are {file_count} files in the directory.')


In [ ]:
import os

# Replace 'your_directory_path' with the path to your target directory
directory_path = '/home/luhan/pairs/QnA_generation_result'

# Replace 'your_file_name' with the name of the file you're looking for
file_name = 'QnA_generation_PMC4142697.json'

# Construct the full file path
file_path = os.path.join(directory_path, file_name)

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file '{file_name}' exists in '{directory_path}'.")
else:
    print(f"The file '{file_name}' does not exist in '{directory_path}'.")


In [ ]:
with open('CC_v2.json', 'r') as f:
    QnA = json.load(f)

In [ ]:
len(QnA)

In [ ]:
with_a = [i for i in QnA if 'A_embeddings' in i]

In [ ]:
without_a = [i for i in QnA if 'A_embeddings' not in i]

In [ ]:
len(with_a)

In [ ]:
len(without_a)

In [ ]:
without_a[:10]

In [ ]:
type(with_a[0]['A_embeddings'])

In [ ]:
import numpy as np

for qna in with_a:
    a_emb = np.array(qna['A_embeddings'])
    q_emb = np.array(qna['Q_embeddings'])
    n = np.dot(a_emb, q_emb)
    qna['similarity'] = n


In [ ]:
with_a[:10]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Replace 'your_list_of_numbers' with your actual list of numbers
your_list_of_numbers = [qna['similarity'] for qna in with_a]
data = np.array(your_list_of_numbers)

# Compute the PDF using a Gaussian Kernel Density Estimation
density = stats.gaussian_kde(data)

# Create a range of values for which we'll plot the PDF
x = np.linspace(min(data), max(data), 1000)

# Plot the PDF
plt.plot(x, density(x))
plt.title('Probability Density Function')
plt.xlabel('Value')
plt.ylabel('Density')

# Show the plot
plt.show()


In [ ]:
after_CC = [i for i in with_a if i['similarity'] > 0.75]

In [ ]:
len(after_CC)

In [ ]:
trial = after_CC[:10][:]

In [ ]:
with open('after_CC.json', 'w') as f:
    json.dump(after_CC, f)

In [ ]:
with open('after_CC.json', 'r') as f:
    after_CC = json.load(f)

In [ ]:
import weaviate.classes as wvc
import weaviate

weaviate.classes.init.Auth.client_password('tiy', 'try.1206')
client = weaviate.connect_to_local(
    host='localhost',
    port=8080,
    grpc_port=50051,

)
collection = client.collections.get('validation')

In [ ]:
import re
from tqdm import tqdm

In [ ]:
rst = []
for qa in tqdm(trial):
    ans = re.search(r'\d+', qa['answer'])
    ans_index = int(ans.group()) - 1
    ans_exp = qa['options'][ans_index] + ' ' + qa['explanation']
    response = collection.query.hybrid(
        query=ans_exp,
        filters=wvc.query.Filter.by_property("collection_id").equal('PMC' + qa['pmc_id']) 
                & wvc.query.Filter.by_property("chunk_id").equal(qa['chunk_id']),
        fusion_type=wvc.query.HybridFusion.RELATIVE_SCORE,
        alpha=0.8,
        vector=qa['A_embeddings'],
        query_properties=["text"],
        return_metadata=wvc.query.MetadataQuery(score=True, explain_score=True),
        limit=1
    )
    qa['matched_text'] = response.objects[0].properties['text']
    qa['matched_score'] = response.objects[0].metadata.score
    rst.append(qa)

In [ ]:
haha = [{k: item[k] for k in ['question', 'answer', 'options', 'explanation', 'matched_text', 'matched_score', 'pmc_id', 'chunk_id']} for item in rst]


In [ ]:
with open('haha.json', 'w') as f:
    json.dump(haha, f)

In [ ]:
with open('../with_text_and_score.json', 'r') as f:
    qq = json.load(f)

In [ ]:
qq[0]

In [ ]:
import os
os.getcwd()

In [ ]:
len(qq)

In [ ]:
qq[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Replace 'your_list_of_numbers' with your actual list of numbers
your_list_of_numbers = [qna['total_score'] for qna in qq]
data = np.array(your_list_of_numbers)

# Compute the PDF using a Gaussian Kernel Density Estimation
density = stats.gaussian_kde(data)

# Create a range of values for which we'll plot the PDF
x = np.linspace(min(data), max(data), 1000)

# Plot the PDF
plt.plot(x, density(x))
plt.title('Probability Density Function')
plt.xlabel('Value')
plt.ylabel('Density')

# Show the plot
plt.show()


In [ ]:
for k in qq:
    k['total_score'] = (k['matched_score'] + k['similarity']) / 2

In [ ]:
sorted_qna = sorted(qq, key=lambda x: x['total_score'])

In [ ]:
sorted_qna[0]

In [ ]:
def divide_list_into_four_parts(lst, n):
    # Calculate the length of each part
    part_length = len(lst) // n

    # Calculate the number of parts that will have an extra element
    extra_elements = len(lst) % n

    parts = []
    start_index = 0

    # Create 4 parts
    for i in range(n):
        # Calculate end index for the current part
        # Add an extra element to the first 'extra_elements' parts
        end_index = start_index + part_length + (1 if i < extra_elements else 0)

        # Slice the list from start_index to end_index and add to the parts list
        parts.append(lst[start_index:end_index])

        # Update the start_index for the next part
        start_index = end_index

    return parts


In [ ]:
parts = divide_list_into_four_parts(sorted_qna, 10)

In [ ]:
len(parts[0])

In [ ]:
# Assuming list1, list2, list3, and list4 are your four lists

def get_boundaries(lst):
    if lst:  # Check if the list is not empty
        # The first element's total_score
        start_value = lst[0]['total_score']
        # The last element's total_score
        end_value = lst[-1]['total_score']
        return start_value, end_value
    else:
        return None, None  # In case the list is empty

# Get boundary values for each list
boundaries_list1 = get_boundaries(parts[0])
boundaries_list2 = get_boundaries(parts[1])
boundaries_list3 = get_boundaries(parts[2])
boundaries_list4 = get_boundaries(parts[3])


# Print the boundary values
print(f"List 1 Boundaries: {boundaries_list1}")
print(f"List 2 Boundaries: {boundaries_list2}")
print(f"List 3 Boundaries: {boundaries_list3}")
print(f"List 4 Boundaries: {boundaries_list4}")


In [ ]:
for p in parts:
    for pr in p:
        del pr['similarity']
        del pr['matched_score']
        del pr['A_embeddings']
        del pr['Q_embeddings']

In [ ]:
len(parts)

In [ ]:
with open('../complete_questions.json', 'w') as f:
    json.dump(parts, f)

In [ ]:
import random

rst = []

for i in range(4):
    drawn_elements = random.sample(parts[i], 250)
    rst.extend(drawn_elements)

In [ ]:
len(rst)

In [ ]:
haha = [{k: item[k] for k in ['question', 'answer', 'options', 'explanation', 'matched_text', 'total_score', 'pmc_id', 'chunk_id']} for item in rst]

In [ ]:
with open('final_1000.json', 'w') as f:
    json.dump(haha, f)